In [4]:
#%save MonCoreNLP.py _ih[2]

In [2]:
"""

serveur dans (home simph)  ./nltk-data/stanford-corenlp-full-2016-10-31/

lancement

java -mx6g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9009 -timeout  15000

warning :

il a besoin de 6G 5g de  ram pour traiter 2 langues comme en et fr
- mettre dans la directory les models java pour chaque langue....voir le site stanford 
coreNLP server.


"""



import time
import requests, logging, sys
import pprint, json, time
import codecs

root = logging.getLogger('Root')
root.setLevel(logging.WARNING)

lhandler = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter(
                '%(asctime)s [%(levelname)s] : %(message)s',
                '%Y-%m-%d %H:%M:%S')
lhandler.setFormatter(formatter)
root.addHandler(lhandler)

class CoreNLP:
    root.debug('Object instantiating..')
    annotator_full_list = ["tokenize", "cleanxml", "ssplit", "pos", 
    "lemma", "ner", "regexner", "truecase", "parse", "depparse", "dcoref", 
    "relation", "natlog", "quote", "sentiment"]
    language_full_list = ['en', 'fr', 'ar', 'zh', 'de', 'es']
    url = 'http://127.0.0.1:9009'
    out_format = 'json'

    def __init__(self, url=url,
                 language = "en",
                 annotator_list=annotator_full_list,
                isPrint = False,
                ):
        
        if isPrint :
            root.setLevel(logging.DEBUG)
            
        
        assert url.upper().startswith('HTTP'), \
            'url string should be prefixed with http'
        
        if 'SENTIMENT' in map(str.upper, annotator_list):
            root.warning('You are using "Sentiment" annotator which is'\
                'not supported by Old version of CoreNLP')
        
        
        if url.endswith('/'):
            self.url = url[:-1]
        else:
            self.url = url

        assert isinstance(annotator_list, list), \
            'annotators can be passed only as a python list'
        if len(annotator_list) == 14:
            root.info('Using all the annotators, It might take a while')
        
        self.annotator_list = annotator_list
        self.language = language
        self.isPrint = isPrint
        common=set(self.annotator_list).intersection(self.annotator_full_list)
        not_suprtd_elem = set(self.annotator_list) - common
        assertion_error = 'annotator not supported: ' + str(not_suprtd_elem)
        assert not not_suprtd_elem, assertion_error
        
        common = set([self.language]).intersection(self.language_full_list)
        not_existing_language = set ( [self.language ]) - common
        assertion_error = 'language not supported: ' + str(not_existing_language)
        assert not not_existing_language, assertion_error
        
        


    @staticmethod
    def server_connection(current_url, data,):
        root.debug('server connection: ' + current_url)
        # modif PL
        root.debug ("data :"+ data )
        #print (type(data))
        assert isinstance(data, unicode) and data, 'Enter valid string input'
        try:
            data_utf = data.encode('utf-8')
        except:
            print (repr(data))
            raise
        try:
            server_out = requests.post(current_url, 
                                        data_utf, 
                                        headers={'Connection': 'close'})
        except requests.exceptions.ConnectionError:
            root.error('Connection Error, check you have server running')
            raise Exception('Check your CoreNLP Server status \n'
                'if not sure, Check the pywrap doc for Server instantiation')
        return server_out
    
   
    def url_calc(self, serializer=''):
        s_string = '/?properties={"annotators": "'
        anot_string = ','.join(self.annotator_list)
        m_string = '", "outputFormat": "' + self.out_format
        f_string = '", "serializer": "' + serializer + '"}&pipelineLanguage='+self.language
        return self.url + s_string + anot_string + m_string + f_string


    def basic(self, data, out_format='json', serializer=''):
        self.out_format = out_format
        format_list = ['JSON', 'XML', 'TEXT', 'SERIALIZED']
        assert out_format.upper() in format_list, \
            'output format not supported, check stanford doc'
        
        if out_format.upper() == 'SERIALIZED' and not serializer:
            root.info(
                'Default Serializer is using - ' + 
                'edu.stanford.nlp.pipeline.ProtobufAnnotationSerializer')
            serializer = ('edu.stanford.nlp.pipeline.'
                'ProtobufAnnotationSerializer')
                
        current_url = self.url_calc(serializer)
        
        
        return self.server_connection(current_url, data,)

    @staticmethod
    def tokensregex(data, pattern='', custom_filter=''):
        root.info('TokenRegex started')
        return CoreNLP.regex('/tokensregex', data, pattern, custom_filter)

    @staticmethod
    def semgrex(data, pattern='', custom_filter=''):
        root.info('SemRegex started')
        return CoreNLP.regex('/semgrex', data, pattern, custom_filter)

    @staticmethod
    def tregex(data, pattern='', custom_filter=''):
        root.info('Tregex started')
        return CoreNLP.regex('/tregex', data, pattern, custom_filter)

    @classmethod
    def regex(cls, endpoint, data, pattern, custom_filter):
        url_string = '/?pattern=' + str(pattern) +'&filter=' + custom_filter 
        current_url = cls.url + endpoint + url_string
        root.info('Returning the data requested')
        return cls.server_connection(current_url, data)

    @staticmethod
    def process_sentences(sentences):
        assert isinstance(sentences, list), 'it should be a list'
        index = 0
        new_index = 0
        token_dict = {
        'index':[],
        'truecaseText':[],
        'ner':[],
        'before':[],
        'originalText':[],
        'characterOffsetBegin':[],
        'lemma':[],
        'truecase':[],
        'pos':[],
        'characterOffsetEnd':[],
        'speaker':[],
        'word':[],
        'after':[],
        'normalizedNER':[]
        }
        for sentence in sentences:
            index = new_index
            tokens = sentence['tokens']
            for val in tokens:

                #workaround to handle length inconsistancie with normalizedNER, rethink the logic
                if 'ner' in val.keys() and 'normalizedNER' not in val.keys():
                    token_dict['normalizedNER'].append(0)
                    
                for key, val in val.items():
                    if key == 'index':
                        new_index = index + int(val)
                        token_dict[key].append(str(new_index))
                    else:
                        try:
                            token_dict[key].append(val)
                        except KeyError:
                            token_dict[key] = [val]
                            root.info('New key added: ' + key)
        
                         
        return [token_dict, sentences]


    def arrange(self, data):
        root.info('Executing custom function')
        assert isinstance(data, unicode) and data, 'Enter valid string input'
        if 'lemma' not in self.annotator_list:
            self.annotator_list.append('lemma')
        
        current_url = self.url_calc()
        r = self.server_connection(current_url, data)
        try:
            r = r.json()
            rs = r['sentences']
        except ValueError:
            root.error('Value Error: '+r.text+', Check special chars in input')
            rs = []
        return self.process_sentences(rs)
    
listePOS = """

        CC Coordinating conjunction
        CD Cardinal number
        DT Determiner
        EX Existential there
        FW Foreign word
        IN Preposition or subordinating conjunction
        JJ Adjective
        JJR Adjective, comparative
        JJS Adjective, superlative
        LS List item marker
        MD Modal
        NN Noun, singular or mass
        NNS Noun, plural
        NNP Proper noun, singular
        NNPS Proper noun, plural
        PDT Predeterminer
        POS Possessive ending
        PRP Personal pronoun
        PRP$ Possessive pronoun
        RB Adverb
        RBR Adverb, comparative
        RBS Adverb, superlative
        RP Particle
        SYM Symbol
        TO to
        UH Interjection
        VB Verb, base form
        VBD Verb, past tense
        VBG Verb, gerund or present participle
        VBN Verb, past participle
        VBP Verb, non­3rd person singular present
        VBZ Verb, 3rd person singular present
        WDT Wh­determiner
        WP Wh­pronoun
        WP$ Possessive wh­pronoun
        WRB Wh­adverb
        """


full_annotator_list = ["ner", "pos", "lemma",  "depparse",  "relation"]

language ="fr"

cf = CoreNLP(url='http://localhost:9009',
             language = language,
             annotator_list=full_annotator_list,
             isPrint = False )

"""
#Calling basic function which would return a 'requests' object
out = cf.basic(data, out_format='json'  )
print ('Basic')haque 
pp.pprint(out.json())
"""




pp = pprint.PrettyPrinter(indent=4) 

full_annotator_list = ["tokenize", "cleanxml", "ssplit", "pos", "lemma", "ner", "regexner", "truecase", "parse",
                       "depparse", "dcoref", "relation", "natlog", "quote"]

full_annotator_list = ["ner", "pos", "lemma",  "depparse",  "relation"]

language ="fr"

cf = CoreNLP(url='http://localhost:9009',
             language = language,
             annotator_list=full_annotator_list,
             isPrint = False )

def get_enhancedPlusPlusDependencies (data) :
    t0 = time.time()
    Arrange, sentences = cf.arrange(data,)
    #print ("temps de calcul = %1.2f secondes" %(time.time() - t0))
    #print ("\n ###################### ==== Affichage du dictionnaire Arrange ===== ###################### \n")
    #pp.pprint (Arrange)
    ner = Arrange ['ner']
    roles = Arrange ['pos']
    words = Arrange ['word']
    #print ("ner" + str(ner))
    
    #print ("\n ###################### ==== Affichage du dictionnaire sentences ===== ###################### \n")
    #pp.pprint (sentences)
    enhancedPlusPlusDependencies = sentences[0] ['enhancedPlusPlusDependencies']
    #print ("\n ## == Affichage de la valeur associée à la clé enhancedPlusPlusDependencies dans le dictionnaire sentences == ## \n")
    #pp.pprint(enhancedPlusPlusDependencies)
    
    return enhancedPlusPlusDependencies, ner , roles, words
    



def lecture_fichier(path = "./definition de famille (simple).txt") :
    data = []
    with codecs.open(path, "r", "utf-8") as f :
        for line in f.readlines():
            if not line.strip().startswith("#") and not "#--" in line.strip() and not line.strip() == "":
                data.append(line.strip("\r\n"))

    f.close()
    return data





In [29]:
#%save affichage.py _ih[27]

In [34]:


def creationArbreDependancePosition (enhancedPlusPlusDependencies) :
    arbre = {}
    for group in enhancedPlusPlusDependencies :
        dep = group [0] # 'dep' == 0
        governor = group [3] #'governor' == 3
        dependent = group [1] # 'dependent' == 1
      
        try:
            arbre [governor].append((dependent, dep))
        except:
            arbre [governor]= [(dependent, dep),]
        continue
    return arbre



def creationArbreDependanceNom (enhancedPlusPlusDependencies) :
    arbre = {}
    for group in enhancedPlusPlusDependencies :
        dep = group [0] # 'dep' == 0
        governorGloss = group [4] # 'governorGloss' == 4
        dependentGloss = group [2] # 'dependentGloss' == 2
        #print("aaaaaaaaaaaaaaaaa", governorGloss)
        try:
            arbre [governorGloss].append((dependentGloss, dep))
        except:
            arbre [governorGloss]= [(dependentGloss, dep),]
        continue
    return arbre

arbre = creationArbreDependanceNom (enhancedPlusPlusDependencies)




#premierMot = "ROOT"



def affichageArbre (mot, arbre,memoire = {}) :
    try:
        listeDependence = arbre[mot]
    except :
        return
    
    for dependentGloss, dep in listeDependence :
        print (mot, "=> " , dependentGloss," pour type de dependance: ", dep)
        if not dependentGloss in memoire :
            affichageArbre (dependentGloss, arbre, memoire = memoire)
            memoire [dependentGloss] = True
        continue
    
    return 

#affichageArbre (premierMot, arbre )  



('aaaaaaaaaaaaaaaaa', {u'dep': u'det', u'dependent': 4, u'governorGloss': u'homme', u'governor': 5, u'dependentGloss': u'un'})


TypeError: unhashable type: 'dict'

In [33]:
#%save lecture_data.py _ih[31]

[u'Une personne est un homme ou une femme.', u'Une femme est f\xe9minine.', u'Un homme est masculin.', u'Une famille a parents et enfants.', u'Un parent est un p\xe8re ou une m\xe8re.', u'Un enfant est un fils ou une fille.', u'Un p\xe8re est un homme.', u'Une m\xe8re est une femme.', u'Une jeune-fille est f\xe9minine.', u'Un gar\xe7on est masculin.', u'Un fils est un gar\xe7on ou un homme.', u'Une fille est une jeune-fille ou une femme.']


In [24]:
#%save family.py _ih[22]

In [40]:
import time, codecs
#from lecture_data import lecture_fichier
#from enhanced import get_enhancedPlusPlusDependencies
#from affichage import creationArbreDependanceNom, affichageArbre






###############Une personne est un homme ou une femme.###############


AssertionError: Enter valid string input

In [16]:
"""

Generalisation intemporelle et hors espace......Concept pas objet anime ou inanimé 

un chat est un carnivore

un carnivore est un mangeur(nc) uniquement de viande.

un mangeur est un (objet animé) qui est (mange V)

La regle est de generaliser les qualités du carnivore au chat par exemple

Le chat de Merlin mange de l'herbe.

réponse : impossible un chat est un carnivore donc il ne mange pas de l'herbe.

retour:

Le chat de Merlin avale de l'herbe pour se purger.

réponse OK

mot1 => "mot general" dico car taille faible 
et 
inversement le retour est un Btree car beaucoup de cas possible (exemple homme = 4 milliards d'individus nommés )


le graphe de calcul se fait sur le mot ensuite sur le "mot general" et ainsi de suite...

arret des le premier résultat ?

tous les mots concept peuvent etre potentiellement un généralisation mais pas de boucle?????

Quand un concept se met à avoir une consistance temps et espace alors il deviend une instance par généralisable...

Un mot peut posseder plusieurs significations au sens concept ou instanciation...

Un mot peut avoir plusieurs sens (grouper les sens mais comment ?)

un mot = "dhdhddhdh" 

une liste des sens , chaque sens est une liste d'objet qui est relié aux autres par mot, sens , position


"""

'\n\nGeneralisation intemporelle et hors espace......Concept pas objet anime ou inanimé \n\nun chat est un carnivore\n\nun carnivore est un mangeur(nc) uniquement de viande.\n\nun mangeur est un (objet animé) qui est (mange V)\n\nLa regle est de generaliser les qualités du carnivore au chat par exemple\n\nLe chat de Merlin mange de l\'herbe.\n\nréponse : impossible un chat est un carnivore donc il ne mange pas de l\'herbe.\n\nretour:\n\nLe chat de Merlin avale de l\'herbe pour se purger.\n\nréponse OK\n\nmot1 => "mot general" dico car taille faible \net \ninversement le retour est un Btree car beaucoup de cas possible (exemple homme = 4 milliards d\'individus nommés )\n\n\nle graphe de calcul se fait sur le mot ensuite sur le "mot general" et ainsi de suite...\n\narret des le premier résultat ?\n\ntous les mots concept peuvent etre potentiellement un généralisation mais pas de boucle?????\n\nQuand un concept se met à avoir une consistance temps et espace alors il deviend une instance p

In [7]:
"""

Cas d'équivallences


#######Cas 1
{nom propre 1} ((etre present)|(be present)) {article indéfini + nom singulier } (de | of ) {nom propre 2}

est équivallent à 

{nom propre 2} ((avoir present) | (have present) ) {article indefini + nom singulier} ("désigné(e)+par " | "called, " )  {nom propre 1}

Exemple :

Merlin est un fils de Patrick.

Donne 

Patrick a un fils,nommé Merlin. 

#######Cas 2
(Chaque | Every ) {nom singulier 1} ((avoir present) | (have present)) {article indefini + nom singulier 2}

et

(All | Tous | Toutes) {article defini plural + singular noun plural } (have present)) {article indefini + nom singulier 2}

est équivallent à

{article indefini + nom sigulier 2 } ("is part of every" | "est une partie de chaque" {nom singulier 1}

Exemple :

Every car have an engine.
All the cars have an engine.

equivallent

An engine is a part of every car.

ou

Chaque voiture a un moteur.
Toutes les voitures ont un moteur.

équivallent 

Un moteur est ume partie de chaque voiture.


"""






'\n\nCas d\'équivallences\n\n\n#######Cas 1\n{nom propre 1} ((etre present)|(be present)) {article indéfini + nom singulier } (de | of ) {nom propre 2}\n\nest équivallent à \n\n{nom propre 2} ((avoir present) | (have present) ) {article indefini + nom singulier} ("désigné(e)+par " | "called, " )  {nom propre 1}\n\nExemple :\n\nMerlin est un fils de Patrick.\n\nDonne \n\nPatrick a un fils,nommé Merlin. \n\n#######Cas 2\n(Chaque | Every ) {nom singulier 1} ((avoir present) | (have present)) {article indefini + nom singulier 2}\n\net\n\n(All | Tous | Toutes) {article defini plural + singular noun plural } (have present)) {article indefini + nom singulier 2}\n\nest équivallent à\n\n{article indefini + nom sigulier 2 } ("is part of every" | "est une partie de chaque" {nom singulier 1}\n\nExemple :\n\nEvery car have an engine.\nAll the cars have an engine.\n\nequivallent\n\nAn engine is a part of every car.\n\nou\n\nChaque voiture a un moteur.\nToutes les voitures ont un moteur.\n\néquivalle

In [8]:
"""

Groupement de la connaissance dans la mémoire 


on connait Cas 1
{nom propre 1} ((etre present)|(be present)) {article indéfini + nom singulier } (de | of ) {nom propre 2}

est équivallent à 

{nom propre 2} ((avoir present) | (have present) ) {article indefini + nom singulier} ("désigné(e)+par " | "called, " )  {nom propre 1}

Si 

{nom propre 2} ((avoir present) | (have present) ) {article indefini + nom singulier} ("désigné(e)+par " | "called, "
)  {nom propre 3}
+
{nom propre 2} ((avoir present) | (have present) ) {article indefini + nom singulier} ("désigné(e)+par " | "called, " 
)  {nom propre 1}

Alors 

{nom propre 2} ((avoir present) | (have present) ) {article indefini + nom singulier} ("désigné(e)+par " | "called, "
{ nom propre 3 } (and" | "et") {nom propre 1}

"""


'\n\nGroupement de la connaissance dans la mémoire \n\n\non connait Cas 1\n{nom propre 1} ((etre present)|(be present)) {article indéfini + nom singulier } (de | of ) {nom propre 2}\n\nest équivallent à \n\n{nom propre 2} ((avoir present) | (have present) ) {article indefini + nom singulier} ("désigné(e)+par " | "called, " )  {nom propre 1}\n\nSi \n\n{nom propre 2} ((avoir present) | (have present) ) {article indefini + nom singulier} ("désigné(e)+par " | "called, "\n)  {nom propre 3}\n+\n{nom propre 2} ((avoir present) | (have present) ) {article indefini + nom singulier} ("désigné(e)+par " | "called, " \n)  {nom propre 1}\n\nAlors \n\n{nom propre 2} ((avoir present) | (have present) ) {article indefini + nom singulier} ("désigné(e)+par " | "called, "\n{ nom propre 3 } (and" | "et") {nom propre 1}\n\n'

In [9]:
"""

Raisonnement dans le temps passé vs present (+ futur ? ) / contexte


{nom propre 1} ( (be past) | (être past)) {article defini + nom singulier} of {nom propre 2}

devient

(nom propre 2) (("have not past)| (non plus avoir past) ) {nom singulier} (anymore | )


ET


{nom propre 1} ( (be past) | (être past)) {article defini | indefini + nom singulier} of {nom propre 2}

devient

(nom propre 2) ((have past)| ( avoir past) ) {indefinite article + nom singulier} ("called ,"| "désigné(e)+par) 
{nom propre 1}

"""

'\n\nRaisonnement dans le temps passé vs present (+ futur ? ) / contexte\n\n\n{nom propre 1} ( (be past) | (être past)) {article defini + nom singulier} of {nom propre 2}\n\ndevient\n\n(nom propre 2) (("have not past)| (non plus avoir past) ) {nom singulier} (anymore | )\n\n\nET\n\n\n{nom propre 1} ( (be past) | (être past)) {article defini | indefini + nom singulier} of {nom propre 2}\n\ndevient\n\n(nom propre 2) ((have past)| ( avoir past) ) {indefinite article + nom singulier} ("called ,"| "désigné(e)+par) \n{nom propre 1}\n\n'

In [1]:
"""

Détection des conflits et génération de questions pour résoudre le conflit



(Every | chaque ) {nom singulier 1} ((be present ) | (être present) ) {article indefini + nom singular 2 }
( (or) | (ou)) {article indefini + nom singular 3 }

En conflit avec 

{nom propre 4} ((be present ) | (être present) ) {article indefini + nom singulier 2 }
( (and) | (et) {article indefini + nom singulier 3 }

exemple :

Chaque personne est un homme ou une femme.

Nadia est une femme et un homme.

=> conflit si Nadia est une personne.


Regle pour raisonner :

# inference
(Every | chaque ) {nom singulier 1} ((be present ) | (être present) ) {article indefini + nom singular 2 }
( (or) | (ou)) {article indefini + nom singular 3 }

ET

{nom propre 1 } ((be present ) | (être present) ) {article indefini + nom singulier 1 }

=>

{nom propre 1 } ((be present ) | (être present) ) {article indefini + nom singular 2 }
( (or) | (ou)) {article indefini + nom singular 3 }


# Question 

{nom propre 1 } ((be present ) | (être present) ) {article indefini + nom singular 2 }
( (or) | (ou)) {article indefini + nom singular 3 }

donne la question

((be present ) | (être present) ) {nom propre 1 } {article indefini + nom singular 2 }
( (or) | (ou)) {article indefini + nom singular 3 } ?



# ou est exclusif

((be present ) | (être present) ) {nom propre 1 } {article indefini + nom singular 2 }
( (or) | (ou)) {article indefini + nom singular 3 } ?

reponse :

{nom propre 1 } ((be present negation ) | (être present negation ) ) {article indefini + nom singular 2 }

Conclusion :

{nom propre 1 } ((be present ) | (être present) )  {article indefini + nom singular 3 }


Ou réponse :

{nom propre 1 } ((be present negation ) | (être present negation ) ) {article indefini + nom singular 3 }

Conclusion :

{nom propre 1 } ((be present ) | (être present) )  {article indefini + nom singular 2 }


"""

'\nDetection des conflits et generation de questions pour resoudre le conflit\n\n\n\n(Every | chaque ) {nom singulier 1} ((be present ) | (être present) ) {article indefini + nom singular 2 }\n( (or) | (ou)) {article indefini + nom singular 3 }\n\nEn conflit avec \n\n{nom propre 4} ((be present ) | (être present) ) {article indefini + nom singular 2 }\n( (and) | (et) {article indefini + nom singular 3 }\n\nexemple :\n\nChaque personne est un homme ou une femme.\n\n\n\n\n\n\n'

In [ ]:
"""

Archivage dans le temps des faits 
(généralisation avec une variable temps dans le contexte, par défaut le temps de la machine )

{nom propre 1} ((etre present)|(be present)) {article défini + nom singulier } (de | of ) {nom propre 2}

équivallent à 

{nom propre 2} ((avoir present) | (have present) ) {article défini + nom singulier} ("désigné(e)+par "
| "called, " )  {nom propre 1}

Donc 

{nom propre 1} ((etre present)|(be present)) {article indéfini + nom singulier } (de | of ) {nom propre 2}

SUIVI PAR 

{nom propre 3} ((etre present)|(be present)) {article indéfini + nom singulier } (de | of ) {nom propre 2}

CONCLUSION 

{nom propre 2} ((avoir past) | (have past ) )  {article défini + nom singulier}  ("désigné(e)+par "
| "called, " )  {nom propre 1}

ET 

{nom propre 3} ((avoir present) | (have present ) )  {article défini + nom singulier}  ("désigné(e)+par "
| "called, " )  {nom propre 2}


EXEMPLE

Bill Clinton est le president des Etats Unis.

Barack Obama est le president des Etats Unis.

=> 

The United State avait un president nommé, Bill Clinton.

The United State a  un president nommé, Barack Obama.


"""
